# Arxiv Chatbot

Ejemplo de chatbot que incluye la definición y ejecución de herramientas en Anthropic API para consulta de papers en Arxiv.

In [1]:
!wget https://gist.githubusercontent.com/juananpe/b7c1683560faf6b44a4d7184e3218c10/raw/304ff3d7c7d3a98c7abcf009d2705b57d6e9d560/requirements-anthropic.txt -O requirements.txt

--2026-02-11 17:04:15--  https://gist.githubusercontent.com/juananpe/b7c1683560faf6b44a4d7184e3218c10/raw/304ff3d7c7d3a98c7abcf009d2705b57d6e9d560/requirements-anthropic.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      44  --.-KB/s    in 0s      

2026-02-11 17:04:15 (2.90 MB/s) - ‘requirements.txt’ saved [44/44]



In [2]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.2 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=c71a4c95bff4c7c7dd978ca5b3e289fb470eebc8d49d75d4deac66c0e8c0b9f5
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


## Import Libraries

In [3]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic

## Tool Functions

In [4]:
PAPER_DIR = "papers"

La primera herramienta busca artículos relevantes en arXiv según un tema y guarda la información de los artículos en un archivo JSON (título, autores, resumen, URL del artículo y fecha de publicación). Los archivos JSON se organizan por temas en el directorio `papers`. La herramienta no descarga los artículos.

In [5]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.

    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)

    Returns:
        List of paper IDs found in the search
    """

    # Use arxiv to find the papers
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)

    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info

    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)

    print(f"Results are saved in: {file_path}")

    return paper_ids

In [6]:
search_papers("Agents")

Results are saved in: papers/agents/papers_info.json


['2203.08975v2',
 '2305.09349v1',
 '2503.19334v1',
 '2112.03763v2',
 '2602.04518v1']

La segunda herramienta extrae información sobre un artículo específico buscando en todos los directorios temáticos dentro del directorio `papers`.

In [ ]:
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.

    Args:
        paper_id: The ID of the paper to look for

    Returns:
        JSON string with paper information if found, error message if not found
    """

    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue

    return f"There's no saved information related to paper {paper_id}."

In [ ]:
extract_info('2501.06243v1')

'{\n  "title": "Agent TCP/IP: An Agent-to-Agent Transaction System",\n  "authors": [\n    "Andrea Muttoni",\n    "Jason Zhao"\n  ],\n  "summary": "Autonomous agents represent an inevitable evolution of the internet. Current\\nagent frameworks do not embed a standard protocol for agent-to-agent\\ninteraction, leaving existing agents isolated from their peers. As intellectual\\nproperty is the native asset ingested by and produced by agents, a true agent\\neconomy requires equipping agents with a universal framework for engaging in\\nbinding contracts with each other, including the exchange of valuable training\\ndata, personality, and other forms of Intellectual Property. A purely\\nagent-to-agent transaction layer would transcend the need for human\\nintermediation in multi-agent interactions. The Agent Transaction Control\\nProtocol for Intellectual Property (ATCP/IP) introduces a trustless framework\\nfor exchanging IP between agents via programmable contracts, enabling agents to\\ni

## Tool Schema

Esquema de cada herramienta que proporcionaremos al LLM.

In [ ]:
tools = [
    {
        "name": "search_papers",
        "description": "Search for papers on arXiv based on a topic and store their information.",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search for"
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "Search for information about a specific paper across all topic directories.",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to look for"
                }
            },
            "required": ["paper_id"]
        }
    }
]

## Tool Mapping

Este código maneja el mapeo y la ejecución de herramientas.

In [ ]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):

    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."

    elif isinstance(result, list):
        result = ', '.join(result)

    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)

    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## Chatbot Code

El chatbot responde a las consultas del usuario una por una, pero no mantiene memoria entre las consultas.

In [ ]:
# load_dotenv()
from google.colab import userdata

os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
client = anthropic.Anthropic()

### Query Processing

In [ ]:
def process_query(query):

    messages = [{'role': 'user', 'content': query}]

    response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219',
                                  tools = tools,
                                  messages = messages)

    process_query = True
    while process_query:
        assistant_content = []

        for content in response.content:
            if content.type == 'text':

                print(content.text)
                assistant_content.append(content)

                if len(response.content) == 1:
                    process_query = False

            elif content.type == 'tool_use':

                assistant_content.append(content)
                messages.append({'role': 'assistant', 'content': assistant_content})

                tool_id = content.id
                tool_args = content.input
                tool_name = content.name
                print(f"Calling tool {tool_name} with args {tool_args}")

                result = execute_tool(tool_name, tool_args)
                messages.append({"role": "user",
                                  "content": [
                                      {
                                          "type": "tool_result",
                                          "tool_use_id": tool_id,
                                          "content": result
                                      }
                                  ]
                                })
                response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219',
                                  tools = tools,
                                  messages = messages)

                if len(response.content) == 1 and response.content[0].type == "text":
                    print(response.content[0].text)
                    process_query = False

### Chat Loop

In [ ]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break

            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

Prueba a interactuar con el chatbot. Aquí tienes un ejemplo de consulta:
- Search for 2 papers on "LLM Jailbreaking"
(o en castellano "Busca 2 artículos sobre "LLM Jailbreaking")


In [ ]:
chat_loop()

Type your queries or 'quit' to exit.

Query: Busca 2 artículos sobre "LLM Jailbreaking"
Puedo buscar artículos sobre "LLM Jailbreaking" para ti. Voy a utilizar la herramienta de búsqueda para encontrar 2 artículos sobre este tema en arXiv.
Calling tool search_papers with args {'topic': 'LLM Jailbreaking', 'max_results': 2}
Results are saved in: papers/llm_jailbreaking/papers_info.json
Ahora obtendré información detallada sobre estos dos artículos:
Calling tool extract_info with args {'paper_id': '2405.20015v2'}
Calling tool extract_info with args {'paper_id': '2312.04127v2'}
Aquí te presento los 2 artículos sobre "LLM Jailbreaking" que encontré:

### Artículo 1
- **Título**: "Efficient Indirect LLM Jailbreak via Multimodal-LLM Jailbreak"
- **Autores**: Zhenxing Niu, Yuyao Sun, Haoxuan Ji, Zheng Lin, Haichang Gao, Xinbo Gao, Gang Hua, Rong Jin
- **Fecha de publicación**: 30 de mayo de 2024
- **Resumen**: Este artículo se centra en los ataques de jailbreak contra grandes modelos de lengu

## Resources

[Guide on how to implement tool use with Claude](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#how-to-implement-tool-use)